In [0]:
# install OpenCV
!pip install opencv-python==3.4.2.16
!pip install opencv-contrib-python==3.4.2.16

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt
import cv2 
import matplotlib.pyplot as plt
import os
from google.colab.patches import cv2_imshow
import glob
import numpy.linalg as la

import keras
import numpy as np 
from keras.models import load_model
from PIL import Image, ImageOps
# from skimage.transform import resize

Using TensorFlow backend.


In [4]:
video = cv2.VideoCapture('angry2.mp4')

face_cascade = cv2.CascadeClassifier('drive/My Drive/Colab Notebooks/haarcascade_frontalface_default.xml')
image_folder = '/project420/image/'
if not os.path.exists(image_folder):
    os.makedirs(image_folder)

count = 0
status = True
font = cv2.FONT_HERSHEY_SIMPLEX 
fontScale = 1
color = (0, 255, 255) 
thickness = 2

# load trained model and reuse it to predict
model = load_model('weights7.12-0.61.h5')

while(status):
    status, frame = video.read()
    if(status == False):
        break
  
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.25, 4)
    # Display the resulting frame
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        # code for only draw box on face
        #  gray_box = gray[y:y+h, x:x+w]
        #  color_box = img[y:y+h, x:x+w]  
        #  plt.imshow(cv2.cvtColor(color_box, cv2.COLOR_BGR2RGB))
        # result = 
        location = (faces[0][0]-5,faces[0][1]-5)

        # Cut the facial part we need for emotion prediction
        test_img = gray[y:y+h+1,:][:,x:x+w+1]
        # first convert the numpy array to an L image,
        # then resize it to 48x48, and convert it back to numpy array
        # in the format required by the trained CNN model
        # for a better resized result
        img = Image.fromarray(test_img,"L")
        size = (48, 48)
        resized_test_gray = ImageOps.fit(img, size, Image.ANTIALIAS)
        resized_test_gray = np.array(resized_test_gray) / 255.0
        test_gray_resized = resized_test_gray.reshape(1, 48, 48, 1)
        # put the face into model and make emotion prediction
        test_pred = np.argmax(model.predict(test_gray_resized))
        # category with highest predicted score will become the output
        label_list = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
        pred = label_list[test_pred]

        # predicted emotion as a string will be printed above the frame
        frame = cv2.putText(frame, pred, location, font,  
                fontScale, color, thickness, cv2.LINE_AA) 

    cv2.imwrite(os.path.join(image_folder, str(count)+'.png'), frame)
    count+=1


In [0]:
# code for making output video
img_array = [None] * count
different_frames = 0
dictionary = {}
for filename in glob.glob('/project420/image/*.png'):
    img = cv2.imread(filename)
    index = int(filename.split("/")[-1].split(".")[0])
    height, width, channels = img.shape
    size = (width,height)
    img_array[index] = img
  
out = cv2.VideoWriter('result_angry2_w16.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [0]:
# code for drawing box at image
img = cv2.imread('/content/face.png')  
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.25, 3)
print(faces)
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    gray_box = gray[y:y+h, x:x+w]
    color_box = img[y:y+h, x:x+w]   
      
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [0]:
# cleaning generated image folders 
!rm -rf /project420/image/